In [1]:
import os
import sys
from pathlib import Path

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Unzip the file (old) - Don't run if already run extract.sh

In [ ]:
zipfiles = !ls | grep ".zip"

In [ ]:
# importing required modules
from zipfile import ZipFile

def pyunzip(file_name):
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # printing all the contents of the zip file
        zip.printdir()

        # extracting all the files
        print('Extracting all the files now...')
        zip.extractall()
        print('Done!')

In [ ]:
dontuse = ['velodyne_points', 'image_00', 'image_01']
for zf in zipfiles:
    !unzip {zf}
    where = "{}/{}".format(zf[:10], zf[:-4])
    for du in dontuse:
        !rm -r {where+'/'+du}

In [ ]:
zipfiles[0][:-4]

# Examinating one of the extracted zip

In [ ]:
ls

In [3]:
roots = Path('.').iterdir()

roots=list(roots)

roots = [root for root in roots if root.is_dir() and root.name!='.ipynb_checkpoints']

roots

[PosixPath('2011_09_29'),
 PosixPath('2011_09_30'),
 PosixPath('2011_09_26'),
 PosixPath('2011_09_28'),
 PosixPath('2011_10_03')]

In [4]:
root = np.random.choice(roots)

folder = np.random.choice( tuple(root.iterdir()) ); folder

PosixPath('2011_09_28/2011_09_28_drive_0134_sync')

In [5]:
img02 = folder /'image_02'
img03 = folder /'image_03'

In [6]:
os.listdir(str(img02))

['timestamps.txt', 'data']

In [7]:
img02d = img02/'data'
img03d = img03/'data'

In [8]:
img02ds = sorted(os.listdir(str(img02d)))
img03ds = sorted(os.listdir(str(img03d)))

In [9]:
tuple(zip( img02ds[:5], img03ds[:5]))

(('0000000000.png', '0000000000.png'),
 ('0000000001.png', '0000000001.png'),
 ('0000000002.png', '0000000002.png'),
 ('0000000003.png', '0000000003.png'),
 ('0000000004.png', '0000000004.png'))

In [10]:
assert img02ds == img03ds

In [11]:
with open(str(img02/'timestamps.txt')) as f:
    print(f.read()[:200])

2011-09-28 13:49:44.755371520
2011-09-28 13:49:44.858636544
2011-09-28 13:49:44.961804288
2011-09-28 13:49:45.065025280
2011-09-28 13:49:45.168167680
2011-09-28 13:49:45.271425536
2011-09-28 13:49:45.


In [12]:
with open(str(img03/'timestamps.txt')) as f:
    print(f.read()[:200])

2011-09-28 13:49:44.754877696
2011-09-28 13:49:44.858016768
2011-09-28 13:49:44.961296640
2011-09-28 13:49:45.064413952
2011-09-28 13:49:45.167686912
2011-09-28 13:49:45.270814464
2011-09-28 13:49:45.


# Visualization

In [ ]:
import matplotlib.animation as animation

In [ ]:
fig, ax = plt.subplots()

shape = plt.imread(str(img02d/img02ds[0])).shape
im = ax.imshow(np.zeros(shape), animated=True)

def frame(paths, root):
    for path in paths:
        yield plt.imread(str(root/path))

def animate(img):
    im.set_array(img)  # update the data
    return im

# Init only required for blitting to give a clean slate.
def init(imgsample):
    im.set_array(np.zeros(shape))
    return im

ani = animation.FuncAnimation(fig, animate, frame(img02ds, img02d), interval=25, blit=True)
plt.show()

In [ ]:
for img in img02ds[:10]:
    plt.imshow(plt.imread(str(img02d/img)))
    plt.show()

In [ ]:
shape = plt.imread(str(img02d/img)).shape; shape

In [ ]:
shape = np.array(shape[:2])

In [ ]:
shape/ shape[0]

In [13]:
def gene_tri(imgs):
    return imgs[0:-2], imgs[1:-1], imgs[2:] 

In [14]:
tris = gene_tri(img02ds)

In [15]:
tris = tuple( zip(*tris) )

In [16]:
tris[0]

('0000000000.png', '0000000001.png', '0000000002.png')

In [ ]:
def plot_tri(tri, path):
    f, axs=plt.subplots(3,1, figsize=(20, 6))
    for i, img in enumerate(tri):
        axs[i].imshow(plt.imread(str(path/img)))
        axs[i].set_axis_off()
    plt.show()

In [ ]:
print(tris[1])
plot_tri(tris[1], img02d)

In [ ]:
def stero_pair(pair, img):
    f, axs=plt.subplots(1,2, figsize=(24, 4))
    for i, loc in enumerate(pair):
        axs[i].imshow(plt.imread(str(loc/img)))
        axs[i].set_axis_off()
    plt.show()

In [ ]:
stero_pair((img02d, img03d), img02ds[0] )

# Convert the directory to csv

In [19]:
roots

[PosixPath('2011_09_29'),
 PosixPath('2011_09_30'),
 PosixPath('2011_09_26'),
 PosixPath('2011_09_28'),
 PosixPath('2011_10_03')]

In [20]:
folder

PosixPath('2011_09_28/2011_09_28_drive_0134_sync')

In [21]:
for root in roots:
    timgs0, timgs1, timgs2 = [], [], []
    folders = root.glob("*_sync")
    for folder in folders:
        img02 = folder /'image_02'/'data'
        img03 = folder /'image_03'/'data'
        
        for root in (img02, img03):
            imgs = sorted([str(img) for img in root.iterdir()])
            imgs0, imgs1, imgs2 = gene_tri(imgs)
            timgs0 += imgs0
            timgs1 += imgs1
            timgs2 += imgs2
        

In [22]:
train = pd.DataFrame(dict(t0=timgs0, t1=timgs1, t2=timgs2)); train.head()

,t0,t1,t2
0,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...
1,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...
2,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...
3,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...
4,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...,2011_10_03/2011_10_03_drive_0058_sync/image_02...


In [23]:
val = train.iloc[int(len(train)*0.9):]
train = train.iloc[:int(len(train)*0.9)]

In [24]:
val.to_csv('validation.csv'); train.to_csv('training.csv');